Para instalar bibliotecas necessárias, use: 
pip install -r requirements.txt

In [ ]:
#Importo bibliotecas que serão usadas
import osmnx as ox
import networkx as nx
import matplotlib.pyplot as plt
import pandas as pd
from pyproj import Transformer, Proj
from geopy.distance import geodesic
import itertools

In [ ]:
#leio arquivos e pego somente colunas uteis!
arqCsv = pd.read_csv("Levantamento ETE_CAGECE - zÉ.csv",usecols=["LATITUDE*","LONGITUDE*"])
arqExcel = pd.read_excel("COMUNIDADES BENEFICIADAS(2).xlsx",usecols=["Latitude","Longitute"])

Transformo o arquivo Excel em uma série de linhas, onde cada linha é uma localização (lat,long)

In [ ]:
#crio função que faz a conversão dos dados do Excel, que estão em UTM para (lat,long)
def TransformaColunasExcel(linha):
    utm_proj =Proj(proj='utm', zone=24, south=True)
    lon, lat =utm_proj(linha.Longitute, linha.Latitude,inverse=True)
    linha.NovaCoord = (round(lat,5),round(lon,5))
    return linha.NovaCoord

#executo função
arqExcel=arqExcel.apply(TransformaColunasExcel ,axis='columns')

Transformo o arquivo CSV em uma série de linhas, onde cada linha é uma localização (lat,long)

In [ ]:
def JuntoColunas(linha):
    return (float(linha["LATITUDE*"].replace(",",".")) ,float(linha["LONGITUDE*"].replace(",",".")))
arqCsv= arqCsv.apply(JuntoColunas ,axis='columns')

Crio tabela com colunas: PontoCsv, PontoExcel para todas as combinações desses valores!

In [ ]:
product = list(itertools.product(arqCsv, arqExcel))
dados = pd.DataFrame(product, columns=['PontoCsv', 'PontoExcel'])

Crio Coluna que contêm a distância geodésica entre os pontos!

In [ ]:
#Vejo se a distância entre os pontos em linha reta é menor do que 50 km
def CalculoGeodesica(linha):
    return geodesic(linha.PontoCsv, linha.PontoExcel).kilometers
dados["DistanciaGeodesica"]= dados.apply(CalculoGeodesica ,axis='columns')
dados.to_csv("ArquivoDistanciaGeodesica.csv",index=False)

Vamos pegar somente linhas onde a distância geodésica for menor que 50 km

In [ ]:
dadosPequenos = dados[dados["DistanciaGeodesica"]<=50]
print(f"Temos: {dadosPequenos.shape[0]} candidatos")
#alinho dados por distância
dadosPequenos=dadosPequenos.sort_values(by=['DistanciaGeodesica'], ascending=[True])
print(dadosPequenos.head())

Agora vamos calcular a distância de carro! Para cada um, pegaremos um mapa e veremos a distância de carro.

In [ ]:
iniciosFeitos =[]
i=0
def CalculoMenorRota(linha):
    global iniciosFeitos
    global i
    i+=1
    print(f"{i} / 1474")
    # Define the coordinates for the center point and the destination point
    center_point = linha.PontoCsv
    destination_point = linha.PontoExcel
    raio = linha.DistanciaGeodesica
    #vejo se já vi essa linha
    if center_point in iniciosFeitos:
        return -1 #significa que já existe caminho!
    else: #caso não tenha um caminho até então
        #Baixo Mapa, multiplica raio por 1000 para ter em km
        G = ox.graph_from_point(center_point, dist=raio*1000, network_type='drive')
        #encontro pontos no mapa mais próximos dos pontos referência
        orig_node = ox.distance.nearest_nodes(G, center_point[1], center_point[0])
        dest_node = ox.distance.nearest_nodes(G, destination_point[1], destination_point[0])
        #encontro menor caminho entre pontos
        shortest_path = nx.shortest_path(G, orig_node, dest_node, weight='length')
        #calculo menor caminho
        path_length = nx.shortest_path_length(G, orig_node, dest_node, weight='length')

        #print(center_point)
        #print(destination_point)
        #print(raio)
        #print(f"The length of the shortest path is {path_length/1000} meters.")

        '''
        fig, ax = ox.plot_graph_route(G, shortest_path, show=False, close=False)
        ax.scatter(center_point[1], center_point[0], c='red', s=100, label='Center Point')
        ax.scatter(destination_point[1], destination_point[0], c='blue', s=100, label='Destination Point')
        plt.legend()
        plt.savefig(f"{i}.jpg")
        plt.show()
        '''
        
        if path_length/1000<=50:
            iniciosFeitos+=[center_point]
        return path_length/1000
    
primeriasLinhas = dadosPequenos.head(5)
primeriasLinhas["Menor Rota em KM"] = primeriasLinhas.apply(CalculoMenorRota ,axis='columns')

Vamos salvar em csv

In [ ]:
primeriasLinhas.to_csv("ArquivoFinal.csv",index=False)